In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.ensemble import IsolationForest


In [19]:
df = pd.read_csv('data/g9_data.csv')

df_g9 = df
df_g9.reset_index(drop=True)

/Users/msaied/mambaforge/envs/default/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (12,13) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,año,eleccion_tipo,eleccion_id,recuento_tipo,recuento_id,padron_tipo,distrito_id,distrito_nombre,seccionprovincial_id,seccionprovincial_nombre,...,mesa_tipo,mesa_electores,cargo_id,cargo_nombre,agrupacion_id,agrupacion_nombre,lista_numero,lista_nombre,votos_tipo,votos_cantidad
0,2023,GENERAL,2,PROVISORIO,1,NORMAL,2,Buenos Aires,7.0,Sección Séptima,...,NATIVOS,346,1,PRESIDENTE Y VICE,134,UNION POR LA PATRIA,NaN,NaN,POSITIVO,98
1,2023,GENERAL,2,PROVISORIO,1,NORMAL,2,Buenos Aires,7.0,Sección Séptima,...,NATIVOS,346,1,PRESIDENTE Y VICE,132,JUNTOS POR EL CAMBIO,NaN,NaN,POSITIVO,71
2,2023,GENERAL,2,PROVISORIO,1,NORMAL,2,Buenos Aires,7.0,Sección Séptima,...,NATIVOS,346,1,PRESIDENTE Y VICE,135,LA LIBERTAD AVANZA,NaN,NaN,POSITIVO,71
3,2023,GENERAL,2,PROVISORIO,1,NORMAL,2,Buenos Aires,7.0,Sección Séptima,...,NATIVOS,346,1,PRESIDENTE Y VICE,133,HACEMOS POR NUESTRO PAIS,NaN,NaN,POSITIVO,13
4,2023,GENERAL,2,PROVISORIO,1,NORMAL,2,Buenos Aires,7.0,Sección Séptima,...,NATIVOS,346,1,PRESIDENTE Y VICE,136,FRENTE DE IZQUIERDA Y DE TRABAJADORES - UNIDAD,NaN,NaN,POSITIVO,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
330054,2023,GENERAL,2,PROVISORIO,1,NORMAL,2,Buenos Aires,2.0,Sección Segunda,...,NATIVOS,93,9,PARLAMENTO MERCOSUR REGIONAL,0,NaN,NaN,NaN,EN BLANCO,13
330055,2023,GENERAL,2,PROVISORIO,1,NORMAL,2,Buenos Aires,2.0,Sección Segunda,...,NATIVOS,93,9,PARLAMENTO MERCOSUR REGIONAL,0,NaN,NaN,NaN,NULO,0
330056,2023,GENERAL,2,PROVISORIO,1,NORMAL,2,Buenos Aires,2.0,Sección Segunda,...,NATIVOS,93,9,PARLAMENTO MERCOSUR REGIONAL,0,NaN,NaN,NaN,IMPUGNADO,0
330057,2023,GENERAL,2,PROVISORIO,1,NORMAL,2,Buenos Aires,2.0,Sección Segunda,...,NATIVOS,93,9,PARLAMENTO MERCOSUR REGIONAL,0,NaN,NaN,NaN,RECURRIDO,0


## Casos Anomalos

Ideas para la deteccion de casos anomalos:
- Comparar resultados en mesas con el resultado general de la seccion
- Comparación entre cargos: Compara los resultados de diferentes cargos (por ejemplo, presidente, diputados y senadores) en la misma elección. Las discrepancias significativas entre los resultados de diferentes cargos pueden ser señales de problemas en ciertas mesas o secciones.
- Comparacion con mesas adyacentes dentro de la seccion
- Participación inusual: Observa la participación electoral. Si hay mesas con tasas de participación inusualmente altas o bajas, esto podría indicar irregularidades, como el acarreo de votantes o la supresión de votos.

Con datasets externos:
- Comparacion con PASO
- Comparacion con elecciones anteriores

In [14]:
# Creamos un dataframe igual al original pero vacio
mesas_anomalas = pd.DataFrame(columns=df_g9.columns)

In [20]:
# Mesas incosistentes. Con mas votos que electores
# Vamos a eliminarlas del dataset

# Agrupamos por mesa y cargo y luego sumamos los votos
df_g9['asistencia_mesa'] = df_g9.groupby(['seccion_nombre', 'mesa_id', 'cargo_nombre'])['votos_cantidad'].transform('sum')

# Eliminamos las mesas que tienen mas votos que electores
df_incosistente = df_g9[df_g9['asistencia_mesa'] > df_g9['mesa_electores']]
df_g9 = df_g9[df_g9['asistencia_mesa'] <= df_g9['mesa_electores']]
mesas_anomalas = pd.concat([df_incosistente, mesas_anomalas])
mesas_anomalas

,año,eleccion_tipo,eleccion_id,recuento_tipo,recuento_id,padron_tipo,distrito_id,distrito_nombre,seccionprovincial_id,seccionprovincial_nombre,...,mesa_electores,cargo_id,cargo_nombre,agrupacion_id,agrupacion_nombre,lista_numero,lista_nombre,votos_tipo,votos_cantidad,asistencia_mesa


In [22]:
# Mesas con 0 votantes
# Eliminamos las mesas que tienen mas votos que electores
df_mesas_cerradas = df_g9[df_g9['asistencia_mesa'] == 0]
df_g9 = df_g9[df_g9['asistencia_mesa'] > 0]

df_mesas_cerradas[['mesa_id', 'seccion_nombre']]
mesas_anomalas = pd.concat([df_mesas_cerradas, mesas_anomalas])
mesas_anomalas

,año,eleccion_tipo,eleccion_id,recuento_tipo,recuento_id,padron_tipo,distrito_id,distrito_nombre,seccionprovincial_id,seccionprovincial_nombre,...,mesa_electores,cargo_id,cargo_nombre,agrupacion_id,agrupacion_nombre,lista_numero,lista_nombre,votos_tipo,votos_cantidad,asistencia_mesa
1510,2023,GENERAL,2,PROVISORIO,1,NORMAL,2,Buenos Aires,7.0,Sección Séptima,...,351,1,PRESIDENTE Y VICE,134,UNION POR LA PATRIA,NaN,NaN,POSITIVO,0,0
1511,2023,GENERAL,2,PROVISORIO,1,NORMAL,2,Buenos Aires,7.0,Sección Séptima,...,351,1,PRESIDENTE Y VICE,133,HACEMOS POR NUESTRO PAIS,NaN,NaN,POSITIVO,0,0
1512,2023,GENERAL,2,PROVISORIO,1,NORMAL,2,Buenos Aires,7.0,Sección Séptima,...,351,1,PRESIDENTE Y VICE,132,JUNTOS POR EL CAMBIO,NaN,NaN,POSITIVO,0,0
1513,2023,GENERAL,2,PROVISORIO,1,NORMAL,2,Buenos Aires,7.0,Sección Séptima,...,351,1,PRESIDENTE Y VICE,135,LA LIBERTAD AVANZA,NaN,NaN,POSITIVO,0,0
1514,2023,GENERAL,2,PROVISORIO,1,NORMAL,2,Buenos Aires,7.0,Sección Séptima,...,351,1,PRESIDENTE Y VICE,136,FRENTE DE IZQUIERDA Y DE TRABAJADORES - UNIDAD,NaN,NaN,POSITIVO,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329829,2023,GENERAL,2,PROVISORIO,1,NORMAL,2,Buenos Aires,2.0,Sección Segunda,...,350,9,PARLAMENTO MERCOSUR REGIONAL,0,NaN,NaN,NaN,EN BLANCO,0,0
329830,2023,GENERAL,2,PROVISORIO,1,NORMAL,2,Buenos Aires,2.0,Sección Segunda,...,350,9,PARLAMENTO MERCOSUR REGIONAL,0,NaN,NaN,NaN,NULO,0,0
329831,2023,GENERAL,2,PROVISORIO,1,NORMAL,2,Buenos Aires,2.0,Sección Segunda,...,350,9,PARLAMENTO MERCOSUR REGIONAL,0,NaN,NaN,NaN,IMPUGNADO,0,0
329832,2023,GENERAL,2,PROVISORIO,1,NORMAL,2,Buenos Aires,2.0,Sección Segunda,...,350,9,PARLAMENTO MERCOSUR REGIONAL,0,NaN,NaN,NaN,RECURRIDO,0,0


In [ ]:
# Mesas con una asistencia anormalmente baja o alta
# Supongamos que tienes un DataFrame llamado 'df' con las columnas 'seccion_nombre', 'mesa_id', 'cargo_nombre', 'votos_cantidad' y 'mesa_electores'

# Calcular la asistencia de la mesa como la suma de 'votos_cantidad' por agrupación y cargo
df = df_g9.copy()

df['asistencia_mesa'] = df.groupby(['seccion_nombre', 'mesa_id', 'cargo_nombre'])['votos_cantidad'].transform('sum')

df['asistencia_mesa'] = (df['asistencia_mesa'] / df['mesa_electores']) * 100

# Filtramos mesas con 0 de asistencia
df = df[df['asistencia_mesa'] > 0]

# Filtramos mesas de extranjeros
df = df[df['mesa_tipo'] != 'EXTRANJEROS']

# Calcular el promedio de asistencia seccional
promedio_asistencia_seccional = df.groupby('seccion_nombre')['asistencia_mesa'].mean().reset_index()
promedio_asistencia_seccional = promedio_asistencia_seccional.rename(columns={'asistencia_mesa': 'promedio_asistencia_seccional'})
promedio_asistencia_seccional.head(10)
# Combinar el DataFrame original con el promedio de asistencia seccional
df = df.merge(promedio_asistencia_seccional, on='seccion_nombre')

# Calcular la diferencia entre la asistencia de cada mesa y el promedio seccional
df['diferencia_asistencia'] = df['asistencia_mesa'] - df['promedio_asistencia_seccional']
df.head(10)

# Calcular la desviación estándar de la diferencia de asistencia por sección
desviacion_asistencia_seccional = df.groupby('seccion_nombre')['diferencia_asistencia'].std().reset_index()

desviacion_asistencia_seccional = desviacion_asistencia_seccional.rename(columns={'diferencia_asistencia': 'desviacion_asistencia_seccional'})

# Combinar el DataFrame original con la desviación estándar de asistencia seccional

df = df.merge(desviacion_asistencia_seccional, on='seccion_nombre')

# Calcular el z-score de la diferencia de asistencia de cada mesa

df['z_score_diferencia_asistencia'] = (df['diferencia_asistencia'] - df['desviacion_asistencia_seccional']) / df['desviacion_asistencia_seccional']

# Calcular el z-score de la diferencia de asistencia de cada mesa

df['z_score_diferencia_asistencia'] = (df['diferencia_asistencia'] - df['desviacion_asistencia_seccional']) / df['desviacion_asistencia_seccional']


# Muestro las mesas con z-score absoluto mayor a 3 ordenado por z-score absoluto descendente
df[df['z_score_diferencia_asistencia'].abs() > 3].sort_values(by='z_score_diferencia_asistencia', ascending=False)[['seccion_nombre','mesa_id','asistencia_mesa','promedio_asistencia_seccional','diferencia_asistencia','z_score_diferencia_asistencia']].head(10)
len(df[df['z_score_diferencia_asistencia'].abs() > 3])s

In [5]:
# Veamos cuantas secciones tenemos
df_g9['seccion_nombre'].unique()

array(['Azul', 'Baradero', 'Arrecifes', 'Bolívar', 'Capitán Sarmiento',
       'Carmen de Areco', 'Colón', 'Exaltación de la Cruz',
       'General Alvear', 'La Plata', 'Olavarría', 'Pergamino', 'Ramallo',
       'Rojas', 'Roque Pérez', 'Saladillo', 'Salto',
       'San Andrés de Giles', 'San Antonio de Areco', 'San Nicolás',
       'San Pedro', 'Tapalqué', '25 de Mayo', 'Zárate'], dtype=object)

### Comparar resultados en mesas con el resultado general de la seccion

In [141]:
# Moveremos los valores de votos_tipo para votos no positivos a agrupacion_nombre para agrupar de manera eficiente
df = df_g9.copy()

# Nos quedamos solo con presidente y vice, diputados y senadores
df = df[(df.cargo_nombre == 'PRESIDENTE Y VICE') | (df.cargo_nombre == 'DIPUTADO NACIONAL') | (df.cargo_nombre == 'SENADOR NACIONAL')]
df['agrupacion_nombre'] = df.apply(lambda x: x['agrupacion_nombre'] if x['votos_tipo'] == 'POSITIVO' else x['votos_tipo'], axis=1)

In [ ]:
votos_por_seccion = df.groupby(['seccion_nombre','agrupacion_nombre'])['votos_cantidad'].sum()

total_por_seccion = votos_por_seccion.groupby('seccion_nombre').transform('sum')

porcentaje_por_seccion = (votos_por_seccion / total_por_seccion) * 100

# Crea un nuevo DataFrame con los porcentajes
df_porcentajes = porcentaje_por_seccion.reset_index(name='porcentaje')
df_porcentajes

,seccion_nombre,agrupacion_nombre,porcentaje
0,25 de Mayo,COMANDO,0.139526
1,25 de Mayo,EN BLANCO,3.687463
2,25 de Mayo,FRENTE DE IZQUIERDA Y DE TRABAJADORES - UNIDAD,1.399243
3,25 de Mayo,HACEMOS POR NUESTRO PAIS,4.217660
4,25 de Mayo,IMPUGNADO,0.000000
...,...,...,...
235,Zárate,JUNTOS POR EL CAMBIO,23.588876
236,Zárate,LA LIBERTAD AVANZA,26.311712
237,Zárate,NULO,0.582266
238,Zárate,RECURRIDO,0.025821


In [ ]:
# Chequeemos por ejemplo La Plata
df_plata = df[df['seccion_nombre'] == 'La Plata']

# Calculemos porcentajes por mesa

df_plata['porcentaje'] = df['votos_cantidad'] / df['mesa_electores'] * 100

# A partir de DIFF podemos ver que hay mesas con porcentajes que no coinciden con los de la seccion, y podemos calcular el porcentaje de desviacion de cada mesa respecto de la seccion
df_plata['resultado_seccion'] = df_plata.apply(lambda x: df_porcentajes[df_porcentajes['agrupacion_nombre'] == x['agrupacion_nombre']]['porcentaje'].iloc[0], axis=1)
df_plata['resultado_seccion']

# Calculemos la diferencia porcentual absoluta de los porcentajes de la mesa tomando como referencia los de la seccion
df_plata['diff'] = (df_plata['porcentaje'] - df_plata['resultado_seccion']) / df_plata['resultado_seccion'] * 100

# Veamos las mesas con mayor diferencia tomando solo positivos
df_plata_positivos = df_plata[df_plata['votos_tipo'] == 'POSITIVO']
df_plata_positivos = df_plata_positivos.sort_values(by='diff', ascending=False)

# Mostramos solo la mesa el porcetnaje de la mesa, la agrupacion, el porcentaje de la seccion y la diferencia
df_plata_positivos[['mesa_id','agrupacion_nombre', 'porcentaje','resultado_seccion','diff']].head(10)

<ipython-input-76-498498650aec>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_plata['porcentaje'] = df['votos_cantidad'] / df['mesa_electores'] * 100
<ipython-input-76-498498650aec>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_plata['resultado_seccion'] = df_plata.apply(lambda x: df_porcentajes[df_porcentajes['agrupacion_nombre'] == x['agrupacion_nombre']]['porcentaje'].iloc[0], axis=1)
<ipython-input-76-498498650aec>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

,mesa_id,porcentaje,agrupacion_nombre,resultado_seccion,diff
20112,1298,7.183908,FRENTE DE IZQUIERDA Y DE TRABAJADORES - UNIDAD,1.399243,413.414055
21483,1435,6.849315,FRENTE DE IZQUIERDA Y DE TRABAJADORES - UNIDAD,1.399243,389.501620
20182,1305,6.837607,FRENTE DE IZQUIERDA Y DE TRABAJADORES - UNIDAD,1.399243,388.664865
18493,1136,6.552707,FRENTE DE IZQUIERDA Y DE TRABAJADORES - UNIDAD,1.399243,368.303829
21803,1467,6.497175,FRENTE DE IZQUIERDA Y DE TRABAJADORES - UNIDAD,1.399243,364.335152
23523,1639,6.034483,FRENTE DE IZQUIERDA Y DE TRABAJADORES - UNIDAD,1.399243,331.267806
10313,318,6.017192,FRENTE DE IZQUIERDA Y DE TRABAJADORES - UNIDAD,1.399243,330.032082
20533,1340,6.017192,FRENTE DE IZQUIERDA Y DE TRABAJADORES - UNIDAD,1.399243,330.032082
10903,377,5.965909,FRENTE DE IZQUIERDA Y DE TRABAJADORES - UNIDAD,1.399243,326.367036
17523,1039,5.797101,FRENTE DE IZQUIERDA Y DE TRABAJADORES - UNIDAD,1.399243,314.302820


10588

In [130]:
# Calculemos el porcentaje de asistencia a la mesa

# Calculamos el procentaje que la cantidad de votos representa sobre la cantidad de electores y sumamos una columna al dataframe

df['porcentaje_asistencia'] = df['votos_cantidad'] / df['mesa_electores'] * 100

df_asistencias_por_mesa = df.groupby(['seccion_nombre', 'mesa_id','cargo_nombre'])['porcentaje_asistencia'].sum().reset_index()
df_asistencias_por_mesa.head(30)
df['porcentaje_asistencia'] = df.apply(lambda x: 
    df_asistencias_por_mesa[
        (df_asistencias_por_mesa['mesa_id'] == x['mesa_id']) & 
        (df_asistencias_por_mesa['cargo_nombre'] == x['cargo_nombre']) &
        (df_asistencias_por_mesa['seccion_nombre'] == x['seccion_nombre'])
        ]['porcentaje_asistencia'].iloc[0], axis=1)

# Buscamos mesas donde la asistencia difiere del promedio de la seccion

df['porcentaje_asistencia_seccion'] = df.groupby(['seccion_nombre'])['porcentaje_asistencia'].transform('mean')

df['diff_asistencia'] = df['porcentaje_asistencia_seccion'] - df['porcentaje_asistencia']

# Veamos las mesas con mayor diferencia tomando solo positivos
df = df.sort_values(by='diff_asistencia', ascending=False)

df.sort_values(by='diff_asistencia', ascending=False)[['mesa_id','seccion_nombre','cargo_nombre','agrupacion_nombre', 'porcentaje_asistencia','porcentaje_asistencia_seccion','diff_asistencia']].head(30)

,mesa_id,seccion_nombre,cargo_nombre,agrupacion_nombre,porcentaje_asistencia,porcentaje_asistencia_seccion,diff_asistencia
114169,48,San Antonio de Areco,DIPUTADO NACIONAL,RECURRIDO,0.0,79.116138,79.116138
74663,48,San Antonio de Areco,SENADOR NACIONAL,LA LIBERTAD AVANZA,0.0,79.116138,79.116138
74661,48,San Antonio de Areco,SENADOR NACIONAL,JUNTOS POR EL CAMBIO,0.0,79.116138,79.116138
34060,35,San Antonio de Areco,PRESIDENTE Y VICE,LA LIBERTAD AVANZA,0.0,79.116138,79.116138
74546,35,San Antonio de Areco,SENADOR NACIONAL,LA LIBERTAD AVANZA,0.0,79.116138,79.116138
34062,35,San Antonio de Areco,PRESIDENTE Y VICE,UNION POR LA PATRIA,0.0,79.116138,79.116138
34192,48,San Antonio de Areco,PRESIDENTE Y VICE,JUNTOS POR EL CAMBIO,0.0,79.116138,79.116138
34063,35,San Antonio de Areco,PRESIDENTE Y VICE,HACEMOS POR NUESTRO PAIS,0.0,79.116138,79.116138
114049,35,San Antonio de Areco,DIPUTADO NACIONAL,EN BLANCO,0.0,79.116138,79.116138
114050,35,San Antonio de Areco,DIPUTADO NACIONAL,NULO,0.0,79.116138,79.116138


In [ ]:
.head

### Metodos Estadisticos

In [114]:
# Buscamos anomalias usando la desviacion estandar
tabla_contingencia = df.groupby(['seccion_nombre', 'mesa_id','cargo_nombre', 'agrupacion_nombre'])['votos_cantidad'].sum().reset_index()

tabla_contingencia = pd.pivot_table(tabla_contingencia, values='votos_cantidad', index=['seccion_nombre', 'mesa_id', 'cargo_nombre'], columns=['agrupacion_nombre'], fill_value=0)


# Calcular la desviación estándar por sección y cargo
desviacion_estandar_por_seccion_cargo = tabla_contingencia.groupby(['seccion_nombre', 'cargo_nombre']).apply(lambda group: np.std(group, axis=1))

desviacion_estandar_por_seccion_cargo
# Calcular el umbral para cada sección basado en el 2 * promedio de la desviación estándar de esa sección
umbral_por_seccion = desviacion_estandar_por_seccion_cargo.groupby('seccion_nombre').mean() * 1.5

# Combinar los resultados en una tabla de umbral por sección
umbral_df = umbral_por_seccion.reset_index(name='umbral')

# Aplicar el umbral para detectar anomalías para cada sección
anomalías_por_seccion = {}

for seccion, umbral in umbral_df[['seccion_nombre', 'umbral']].itertuples(index=False):
    mesa_anomalia = desviacion_estandar_por_seccion_cargo.loc[seccion].gt(umbral, axis=0)
    if mesa_anomalia.any():
        anomalías_por_seccion[seccion] = mesa_anomalia

# Mostrar las anomalías por sección
for seccion, anomalías in anomalías_por_seccion.items():
    print(f'Anomalías en la sección {seccion}')
    print(anomalías[anomalías].index.values)
    print()

Anomalías en la sección La Plata
[('DIPUTADO NACIONAL', 'La Plata', 440, 'DIPUTADO NACIONAL')
 ('SENADOR NACIONAL', 'La Plata', 440, 'SENADOR NACIONAL')]



In [113]:
# Buscamos anomalias usando Isolation Forest
from sklearn.ensemble import IsolationForest

# Supongamos que tienes una tabla de contingencia como se describió
df['porcentaje'] = df['votos_cantidad'] / df['mesa_electores'] * 100

tabla_contingencia = df.groupby(['seccion_nombre', 'mesa_id','cargo_nombre', 'agrupacion_nombre'])['porcentaje'].sum().reset_index()

tabla_contingencia = pd.pivot_table(tabla_contingencia, values='porcentaje', index=['seccion_nombre', 'mesa_id', 'cargo_nombre'], columns=['agrupacion_nombre'], fill_value=0)

# Dropear rows con 0 en todas las columnas
tabla_contingencia = tabla_contingencia[(tabla_contingencia.T != 0).any()]

# Crear un DataFrame para almacenar las características utilizadas en la detección de anomalías
caracteristicas = tabla_contingencia.reset_index()

# Inicializar el modelo de Isolation Forest
modelo = IsolationForest(contamination=0.005)  # Puedes ajustar el valor de 'contamination' según la proporción de anomalías esperadas

# Crear un DataFrame para almacenar las anomalías por sección
anomalias_por_seccion = pd.DataFrame(columns=['seccion_nombre', 'mesa_id', 'cargo_nombre', 'anomalia'])

# Realizar la detección de anomalías por sección
for seccion, grupo in caracteristicas.groupby('seccion_nombre'):
    X = grupo.iloc[:, 3:]  # Ignoramos las columnas de sección, mesa y cargo

    # Ajustar el modelo a los datos de la sección actual
    modelo.fit(X)

    # Predecir las anomalías para la sección actual (-1 indica anomalía, 1 indica no anomalía)
    predicciones = modelo.predict(X)

    # Agregar las predicciones al DataFrame de anomalías por sección
    grupo['anomalia'] = predicciones
    anomalias_por_seccion = anomalias_por_seccion.append(grupo, ignore_index=True)

# Filtrar las mesas consideradas como anomalías por sección
mesas_anomalas_por_seccion = anomalias_por_seccion[anomalias_por_seccion['anomalia'] == -1]

# Resultado: 'mesas_anomalas_por_seccion' contendrá las mesas consideradas anomalías por sección según el modelo de Isolation Forest.

mesas_anomalas_por_seccion.to_csv('data/mesas_anomalas_por_seccion.csv', index=False)

13167
12896
